In [2]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from language_tool_python import LanguageTool
from spellchecker import SpellChecker

# Load BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name)
model.eval()

# Load LanguageTool for grammar checking
grammar_tool = LanguageTool('en-US')

def calculate_grammar_score(essay):

    # Check grammar
    matches = grammar_tool.check(essay)

    # Calculate grammar score out of 10
    max_score = 10
    grammar_score = max_score - len(matches)
    
    # Ensure the score is within the valid range
    grammar_score = max(0, min(max_score, grammar_score))
    if grammar_score == 0:
        grammar_score=grammar_score+2
    return round(grammar_score,1)
    

def calculate_spelling_score(essay):
    spell = SpellChecker()
    words = essay.split()
    misspelled = spell.unknown(words)
    
    # Get the raw spelling score
    raw_score = len(misspelled)
    
    # Define the mapping between original range and desired range
    original_range = (0, 30)  # Adjust this based on your expected range of misspelled words
    desired_range = (10, 1)   # Reverse the order to get higher scores for fewer mistakes
    
    # Scale the raw score to the desired range
    scaled_score = (raw_score - original_range[0]) / (original_range[1] - original_range[0]) \
                   * (desired_range[1] - desired_range[0]) + desired_range[0]
    
    # Ensure the scaled score is within the desired range
    scaled_score = max(min(scaled_score, max(desired_range)), min(desired_range))
    
    return round(scaled_score,1)

def calculate_word_diversity(essay):
    # Tokenize the essay into words
    words = essay.split()

    # Calculate the number of unique words
    unique_words = set(words)
    num_unique_words = len(unique_words)

    # Calculate the word diversity score (ratio of unique words to total words)
    word_diversity_score = num_unique_words / len(words) if len(words) > 0 else 0

    # Scale the score to be out of 10
    scaled_word_diversity_score = word_diversity_score * 10

    return round(min(scaled_word_diversity_score, 10),1)
def grade_essay(essay):
    # Tokenize and encode the essay using BERT tokenizer
    inputs = tokenizer(essay, return_tensors="pt", truncation=True, padding=True)
    
    # Get the BERT model prediction
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Assuming the model predicts a score in the range [0, 1] for essay quality
    # Assuming the model predicts scores for multiple classes
    class_probabilities = torch.softmax(outputs.logits, dim=1)

    # You may want to use the class with the highest probability as your essay quality score
    predicted_class = torch.argmax(class_probabilities, dim=1)

    # Assuming your scores are in the range [0, 1]
    essay_quality_score = class_probabilities[0, predicted_class].item() * 10

    # essay_quality_score = outputs.logits.item() * 10  # Scale to a score out of 10
    
    # Calculate scores for each parameter
    grammar_score = calculate_grammar_score(essay)
    spelling_score = calculate_spelling_score(essay)
    word_diversity_score = calculate_word_diversity(essay)
    print(grammar_score)
    print(spelling_score)
    print(word_diversity_score)
    print(essay_quality_score)
    # Combine scores and return the final grade
    total_score = (grammar_score + spelling_score + word_diversity_score) / 3
    return round(essay_quality_score,1)

# Example usage
# essay_to_grade = "Population explosion is one of the most serious problems in the world. Therefore a rule was promulgated to limit the number of children a family can have. In my opinion,it obeys basic human rights,some people really like children and they have financial to bring them up. They should have as many children as they like. Besides,only one child in a family causes some problems.The children always feel lonely,meanwhile parents often give them too much love,which lead them have some bad habits,just like selfish. Some people think population explosion bring some problems.It may increase the earth burden,we need more food to feed people,we need more water,more resources.However,I believe people have enough able to solve those problems. Considering the future of our children,limit the number of children is unwise."
# grade = grade_essay(essay_to_grade)
# print(f"Grade for the essay: {grade}/10")


c:\Users\siddh\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
import pandas as pd
# Read essays from the CSV file
essays_df = pd.read_csv('model2.csv')

# Function to calculate scores and update the DataFrame
def calculate_scores_and_update_df(df):
    # Iterate over each row (essay)
    for index, row in df.iterrows():
        essay = row['Essays']  # Assuming 'essay' is the column name containing the essays
        # Calculate scores
        grammar_score = calculate_grammar_score(essay)
        spelling_score = calculate_spelling_score(essay)
        word_diversity_score = calculate_word_diversity(essay)
        essay_quality_score = grade_essay(essay)
        # Update DataFrame with scores
        df.at[index, 'Grammar Score'] = grammar_score
        df.at[index, 'Spelling Score'] = spelling_score
        df.at[index, 'Word Diversity Score'] = word_diversity_score
        df.at[index, 'Essay Quality Score'] = essay_quality_score

# Apply scoring function and update the DataFrame
calculate_scores_and_update_df(essays_df)

# Save the updated DataFrame to a new CSV file
essays_df.to_csv('model2_final.csv', index=False)

2
1
6.5
6.476115584373474
4
5.5
6.2
6.446475982666016
1
2.5
5.6
6.431941390037537
3
4.9
6.4
6.30523681640625
10
3.4
6.1
6.505116820335388
2
4.0
7.0
6.505624055862427
8
5.5
5.6
6.666485667228699
2
2.8
2.0
7.266813516616821
5
1
6.6
6.286317706108093
9
1.6
7.0
6.105132699012756
6
2.5
6.7
6.51044487953186
10
5.2
6.3
6.659994721412659
8
4.6
6.9
6.248441934585571
7
6.4
6.1
6.366661787033081
6
3.4
6.8
6.418891549110413
9
3.1
6.4
6.324787139892578
4
4.0
6.9
6.452975869178772
6
4.3
6.7
6.560290455818176
9
4.9
6.1
6.579070091247559
3
1.3
6.5
6.411119103431702
6
5.2
6.1
6.567816138267517
5
5.5
6.6
6.373659372329712
2
1
6.8
6.337913274765015
3
5.2
7.2
6.56893253326416
7
5.8
7.4
6.619383692741394
9
3.7
6.5
6.54568076133728
9
5.8
6.6
6.281826496124268
2
2.2
7.1
6.327542662620544
8
4.0
6.5
6.560951471328735
10
4.6
6.4
6.34128212928772
2
6.7
5.0
6.503906846046448
9
5.8
5.1
6.503797173500061
5
1.9
6.0
6.415253281593323
2
4.6
6.8
6.311817169189453
2
5.5
7.6
6.292164325714111
5
4.9
5.9
6.586505770683289


In [1]:
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import datetime
from sklearn.metrics import mean_squared_error

# Load the dataset
df = pd.read_csv("C:/Users/siddh/Desktop/BERT/New folder/model2_final.csv")

# Extract essay texts and labels
essays = df['Essays'].tolist()
labels = df[['Grammar Score','Spelling Score','Word Diversity Score','Essay Quality Score']].values

# Tokenization function
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_text(text):
    tokens = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        truncation=True,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )
    return tokens['input_ids'], tokens['attention_mask']

# Prepare data function
def prepare_data(texts, labels):
    input_ids = []
    attention_masks = []

    for text in texts:
        tokens = tokenize_text(text)
        input_ids.append(tokens[0])
        attention_masks.append(tokens[1])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels, dtype=torch.float32)  # Ensure labels are of the correct type

    return input_ids, attention_masks, labels

# Split the dataset into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(essays, labels, test_size=0.2, random_state=42)

# Define the BERT model for regression on each target
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=4)

optimizer = AdamW(model.parameters(), lr=1e-5)

# Prepare the training data
input_ids, attention_masks, labels = prepare_data(essays, labels)
dataset = TensorDataset(input_ids, attention_masks, labels)
train_dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

# Prepare the validation data
val_input_ids, val_attention_masks, val_labels = prepare_data(val_texts, val_labels)
val_dataset = TensorDataset(val_input_ids, val_attention_masks, val_labels)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=False)

def evaluate_model(model, dataloader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for batch in dataloader:
            inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
            outputs = model(**inputs)
            logits = outputs.logits
            all_preds.extend(logits.cpu().numpy())
            all_labels.extend(batch[2].cpu().numpy())
        mse = mean_squared_error(all_labels, all_preds)
        print(f'Mean Squared Error: {mse}')
    return mse

# Train the model
num_epochs = 12
best_val_loss = float('inf')  # Initialize with a large value for comparison

for epoch in range(num_epochs):
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        outputs = model(**inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    # Evaluate on the validation set
    val_loss = evaluate_model(model, val_dataloader)

    # Print or log the learning rate if desired
    print(f'Epoch {epoch + 1}, Mean Squared Error: {val_loss}')

    # Save the model if validation loss improves
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        model_save_path = f"essay_scoring_model_regression_{timestamp}"
        model.save_pretrained(model_save_path)


c:\Users\siddh\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
c:\Users\siddh\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Mean Squared Error: 22.195011138916016
Epoch 1, Mean Squared Error: 22.195011138916016
Mean Squared Error: 17.267799377441406
Epoch 2, Mean Squared Error: 17.267799377441406
Mean Squared Error: 13.254861831665039
Epoch 3, Mean Squared Error: 13.254861831665039
Mean Squared Error: 10.613543510437012
Epoch 4, Mean Squared Error: 10.613543510437012
Mean Squared Error: 8.51755142211914
Epoch 5, Mean Squared Error: 8.51755142211914
Mean Squared Error: 6.853306770324707
Epoch 6, Mean Squared Error: 6.853306770324707
Mean Squared Error: 5.536619186401367
Epoch 7, Mean Squared Error: 5.536619186401367
Mean Squared Error: 4.5372724533081055
Epoch 8, Mean Squared Error: 4.5372724533081055
Mean Squared Error: 3.8529245853424072
Epoch 9, Mean Squared Error: 3.8529245853424072
Mean Squared Error: 3.4523301124572754
Epoch 10, Mean Squared Error: 3.4523301124572754
Mean Squared Error: 3.308626890182495
Epoch 11, Mean Squared Error: 3.308626890182495
Mean Squared Error: 3.35064435005188
Epoch 12, Mean

In [1]:
from transformers import BertForSequenceClassification, BertTokenizer, AdamW
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
import datetime
from sklearn.metrics import mean_squared_error

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_text(text):
    tokens = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        max_length=512,
        truncation=True,
        return_token_type_ids=False,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='pt'
    )
    return tokens['input_ids'], tokens['attention_mask']


def prepare_data(texts, labels):
    input_ids = []
    attention_masks = []

    for text in texts:
        tokens = tokenize_text(text)
        input_ids.append(tokens[0])
        attention_masks.append(tokens[1])

    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)
    labels = torch.tensor(labels, dtype=torch.float32)  # Ensure labels are of the correct type

    return input_ids, attention_masks, labels

# Load the saved model
model_path = "C:\\Users\\siddh\\Desktop\\BERT\\New folder\\essay_scoring_model_regression_20240229_133324"  # Replace <timestamp> with the actual timestamp
model = BertForSequenceClassification.from_pretrained(model_path)

# Tokenize the input essays
input_essays = ["With the development of our city, human right is talked by everyone of us more and more often, and for the city, two kids allowed to be born, is another stone throw into the lake, and that means the number of people will get a new chance to up to a amazing point, by the way, can we ask us a simple question: what are we going to sacrifice again to tell what's right or what's wrong with us. we only see high speed grow up of our city, but we never see the cut down tree, the populated watered, we can not put up a banner on a lonely tree surrounded by stumps, the go up number of people will lead to many problem, which many person think is the outstanding of human right. It's not what we say but what we that matters, we can't bring the whole world into mad then thinking what we could do to save it."]

# Prepare input tensors
input_ids, attention_masks, _ = prepare_data(input_essays, [])

# Make predictions
model.eval()
with torch.no_grad():
    inputs = {'input_ids': input_ids, 'attention_mask': attention_masks}
    outputs = model(**inputs)
    logits = outputs.logits
    predictions = logits.cpu().numpy()

# Post-process the predictions if necessary
# For example, if you want to get the final predicted scores for each aspect:
predicted_scores = predictions.squeeze()  # Squeeze removes the extra dimension added by batch processing

# Now predicted_scores contains the predicted scores for each aspect
print(predicted_scores)


c:\Users\siddh\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[4.600276  2.621825  6.5486956 5.6218677]
